# Create Multi-AI Agent to Research and Write an Article


Learn about Agents, Tasks and organize agents to work sequentially.

In [1]:
# To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:
import sys
print(sys.version)

3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]


### Step 01. Intalling dependencies

Installing crewAi library and crewAI tools

In [2]:
! pip install crewai

  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-instrumentation-fastapi to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/6.4 MB ? eta -:--:--
   ---------------------------------------  6.3/6.4 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.4 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------- 6.4/6.4 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   -------------------- ------------------- 7.9/15.6 MB 37.4 MB/s eta 0:00:01
   ------------------------------ --------- 11.8/15.6 MB 30.8 MB/s eta 0:00:01
   ------------------------------------ --- 14.4/15.6 MB 23

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ai21 3.0.0 requires tenacity<9.0.0,>=8.3.0, but you have tenacity 9.0.0 which is incompatible.
deepeval 1.5.5 requires grpcio~=1.63.0, but you have grpcio 1.68.0 which is incompatible.
deepeval 1.5.5 requires opentelemetry-api~=1.24.0, but you have opentelemetry-api 1.28.2 which is incompatible.
deepeval 1.5.5 requires opentelemetry-exporter-otlp-proto-grpc~=1.24.0, but you have opentelemetry-exporter-otlp-proto-grpc 1.28.2 which is incompatible.
deepeval 1.5.5 requires opentelemetry-sdk~=1.24.0, but you have opentelemetry-sdk 1.28.2 which is incompatible.
deepeval 1.5.5 requires tenacity~=8.4.1, but you have tenacity 9.0.0 which is incompatible.
llama-index-core 0.11.23 requires tenacity!=8.4.0,<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
llama-index-legacy 0.9.48.post4 requires tenacity<9.0

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Step 02. Importing Libraries

Installing crewAi library and crewAI tools

In [4]:
from crewai import Agent, Task, Crew

For LLM agents, we will be using OpenAI's gpt-3.5-turbo.

In [5]:
import os
import dotenv

dotenv.load_dotenv()

True

### Step 03. Creating Agents

Create instance of the Agent class with the desired properties, `role`, `goal` and `backstory`

- `role`: agent's function within the crew
- `goal`: agent's objective. Guides agent's decision making process
- `backstory`: provides context to agent's role and goal. Helps in interation and collaboration.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama-3.1-8b-instant"
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama-3.1-8b-instant"
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama-3.1-8b-instant"
)

### Step 04. Define the Tasks

Tasks are **specific assignments**, completed by agents. 
They **provide all necessary details for execution**, such as a `description`, `expected_output` , `agent` responsible, tools, etc.

- `description`: Tells about what task entails
- `expected_output`: Tells about what task output should look like
- `agent`: Agent responsible for the task

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer
)

### Task: Edit

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Step 05. Forming the crew

Crew is **group of AI agents**, collaborating to perform set of tasks.


- `tasks`: list of tasks
- `agents`: list of agents
- `process (optional)`: work flow (e.g., sequential, hierarchical) default : sequential 
- `verbose=2` allows you to see all the logs of the execution. 


    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.



In [14]:
crew = Crew(
    tasks=[plan, write, edit],
    agents=[planner, writer, editor],
    verbose=True
)

### Step 06.  Kick It Off

Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action Input:' after 'Action:'. I will do right next, and don't use a tool I have already used.

 

Action 'Plan a comprehensive content plan document on Artificial Intelligence.' don't exist, these are the only available Actions:




# Agent: Content Planner
## Using tool: Plan a comprehensive content plan document on Artificial Intelligence.
## Tool Input: 
"To create a thorough content plan document on Artificial Intelligence, I will follow the steps outlined below:\n\n**Action 1: Prioritize the latest trends, key players, and noteworthy new

To display the result as markdown in the notebook.

In [22]:
from IPython.display import Markdown
Markdown(result.raw)

# The Future of Artificial Intelligence
## Unlocking the Potential of AI for a Brighter Tomorrow

Artificial intelligence (AI) has been a topic of fascination for decades, with its impact on our lives becoming increasingly significant. As we continue to push the boundaries of innovation, it's essential to understand the benefits and challenges of AI and its potential to transform our society. In this article, we'll delve into the world of AI, exploring its history, benefits, challenges, and future prospects.

## The Evolution of Artificial Intelligence

The concept of AI dates back to the 1950s when pioneers like Alan Turing and Marvin Minsky first proposed the idea of creating machines that could think and learn like humans (Boden, 2006; Turing, 1950). Since then, AI has undergone significant advancements in machine learning, deep learning, and natural language processing, becoming an integral part of our daily lives. From virtual assistants to self-driving cars, AI is transforming industries and revolutionizing the way we live.

### The Power of Artificial Intelligence in Healthcare

The benefits of AI in healthcare are undeniable, with AI-powered systems helping doctors diagnose diseases more accurately and develop personalized treatment plans (Kaur et al., 2020). AI has also improved patient outcomes by predicting hospital readmissions and detecting potential complications. Moreover, AI-powered chatbots are simplifying the patient experience by providing access to information and completing administrative tasks.

### The Challenges of Artificial Intelligence

Despite its numerous benefits, AI also presents challenges that must be addressed. Job displacement is a significant concern as AI takes over tasks previously performed by humans (McAfee & Brynjolfsson, 2017). Additionally, AI raises concerns about bias and unfairness, as AI systems can perpetuate existing social biases if they are trained on biased data (Dastin, 2018). It's crucial that we prioritize responsible AI development, addressing these concerns to ensure that AI benefits society as a whole.

## The Future of Artificial Intelligence

The future of AI is exciting and uncertain. As AI continues to evolve, it's expected to revolutionize industries such as education, finance, and transportation. However, it's essential that we invest in AI research and education to create a future where AI enhances our lives without diminishing our humanity. By embracing AI and working together to address its challenges, we can create a brighter tomorrow for all.

## Conclusion

Artificial intelligence has the potential to transform our lives in ways we cannot yet imagine. While AI presents many benefits, it also raises important challenges that must be addressed. By working together and prioritizing responsible AI development, we can unlock the true potential of AI and create a brighter future for all.

### References:

Boden, M. A. (2006). Mind as machine: A history of cognitive science. Wiley.

Dastin, J. (2018). Amazon's AI boss says it's committed to avoiding bias. Reuters.

Kaur, R., Kaushal, H., & Choudhary, A. K. (2020). Artificial intelligence in healthcare: A review. Journal of Medical Systems, 44(10), 1-15.

McAfee, A., & Brynjolfsson, E. (2017). Machine, platform, crowd: The future of work. W.W. Norton & Company.

Turing, A. M. (1950). Computing machinery and intelligence. Mind, 59(236), 433-460.

In [24]:
!pip freeze > requirements.txt